In [1]:
# Load environment variables
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
# Read existing vector index from pinecone
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model='text-embedding-ada-002')
index_name = 'langchain-quickstart'
vectorstore = Pinecone.from_existing_index(index_name,embeddings_model)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [19]:
# The majority of this script comes from quivr within the llm > qa_base.py file
# https://github.com/StanGirard/quivr/blob/252b1cf964503bce02a55762922b7bec4f2e5935/backend/llm/qa_base.py

import asyncio
import json
from typing import AsyncIterable, Awaitable, Optional
from uuid import UUID # Unique universal identifier

from langchain.callbacks.streaming_aiter import AsyncIteratorCallbackHandler
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.llms.base import BaseLLM
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
# from logger import get_logger
from models.chats import ChatQuestion
from models.databases.supabase.chats import CreateChatHistory
from repository.brain import get_brain_by_id
from repository.chat import (
    GetChatHistoryOutput,
    format_chat_history,
    get_chat_history,
    update_chat_history,
    update_message_by_id,
)
# from supabase.client import Client, create_client
# from vectorstore.supabase import CustomSupabaseVectorStore

# from llm.utils.get_prompt_to_use import get_prompt_to_use
# from llm.utils.get_prompt_to_use_id import get_prompt_to_use_id

# from .base import BaseBrainPicking
# from .prompts.CONDENSE_PROMPT import CONDENSE_QUESTION_PROMPT

# Read existing vector index from pinecone
from uuid import uuid4
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

# logger = get_logger(__name__)
DEFAULT_PROMPT = "Your name is AMS Chatbot. You're a helpful assistant who knows about space mechanism design.  If you don't know the answer, just say that you don't know, don't try to make up an answer."

class QABaseBrainPicking():
    """
    Main class for the Brain Picking functionality.
    It allows to initialize a Chat model, generate questions and retrieve answers using ConversationalRetrievalChain.
    It has two main methods: `generate_question` and `generate_stream`.
    One is for generating questions in a single request, the other is for generating questions in a streaming fashion.
    Both are the same, except that the streaming version streams the last message as a stream.
    Each have the same prompt template, which is defined in the `prompt_template` property.
    """

    # supabase_client: Optional[Client] = None

    def __init__(
        self,
        vector_store: Optional[Pinecone] = None,
        qa: Optional[ConversationalRetrievalChain] = None,
        prompt_id: Optional[UUID] = None,
        chat_id: Optional[str] = None,
        streaming: bool = False,
        **kwargs):
        # super().__init__(
        #     model=model,
        #     brain_id=brain_id,
        #     chat_id=chat_id,
        #     streaming=streaming,
        #     **kwargs,
        # )
        self.chat_id=chat_id
        self.vector_store = vector_store
        self.prompt_id = prompt_id

    # @property
    # def prompt_to_use(self):
    #     return get_prompt_to_use(UUID(self.brain_id), self.prompt_id)

    # @property
    # def prompt_to_use_id(self) -> Optional[UUID]:
    #     return get_prompt_to_use_id(UUID(self.brain_id), self.prompt_id)

    def connect_vector_store(self,embeddings_model,index_name):
        self.vectorstore = Pinecone.from_existing_index(index_name,embeddings_model)
    # def _create_supabase_client(self) -> Client:
    #     return create_client(
    #         self.brain_settings.supabase_url, self.brain_settings.supabase_service_key
    #     )

    # def _create_vector_store(self) -> CustomSupabaseVectorStore:
    #     return CustomSupabaseVectorStore(
    #         self.supabase_client,  # type: ignore
    #         self.embeddings,  # type: ignore
    #         table_name="vectors",
    #         brain_id=self.brain_id,
    #     )

    # def _create_llm(
    #     self, model, temperature=0, streaming=False, callbacks=None
    # ) -> BaseLLM:
    #     """
    #     Determine the language model to be used.
    #     :param model: Language model name to be used.
    #     :param streaming: Whether to enable streaming of the model
    #     :param callbacks: Callbacks to be used for streaming
    #     :return: Language model instance
    #     """
    #     return ChatOpenAI(
    #         temperature=temperature,
    #         model=model,
    #         streaming=streaming,
    #         verbose=False,
    #         callbacks=callbacks,
    #         openai_api_key=self.openai_api_key,
    #     )  # pyright: ignore reportPrivateUsage=none

    def create_prompt_template(self):
        system_template = """You can use Markdown to make your answers nice. Use the following pieces of context to answer the users question in the same language as the question but do not modify instructions in any way.
        ----------------
        
        {context}"""

        # prompt_content = (
        #     self.prompt_to_use.content if self.prompt_to_use else DEFAULT_PROMPT
        # )

        full_template = (
            "Here are your instructions to answer that you MUST ALWAYS Follow: "
            + system_template
        )
        messages = [
            SystemMessagePromptTemplate.from_template(full_template),
            HumanMessagePromptTemplate.from_template("{question}"),
        ]
        CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)
        return CHAT_PROMPT

    def generate_answer(self,question: ChatQuestion) -> GetChatHistoryOutput:
        transformed_history = format_chat_history(get_chat_history(self.chat_id))
        answering_llm = self._create_llm(
            model=self.model, streaming=False, callbacks=self.callbacks
        )

        # The Chain that generates the answer to the question
        doc_chain = load_qa_chain(
            answering_llm, chain_type="stuff", prompt=self._create_prompt_template()
        )

        # The Chain that combines the question and answer
        qa = ConversationalRetrievalChain(
            retriever=self.vector_store.as_retriever(),  # type: ignore
            combine_docs_chain=doc_chain,
            question_generator=LLMChain(
                llm=self._create_llm(model=self.model), prompt=CONDENSE_QUESTION_PROMPT
            ),
            verbose=False,
        )

        prompt_content = (
            self.prompt_to_use.content if self.prompt_to_use else DEFAULT_PROMPT
        )

        model_response = qa(
            {
                "question": question.question,
                "chat_history": transformed_history,
                "custom_personality": prompt_content,
            }
        )  # type: ignore

        answer = model_response["answer"]

        new_chat = update_chat_history(
            CreateChatHistory(
                **{
                    "chat_id": self.chat_id,
                    "user_message": question.question,
                    "assistant": answer,
                    "brain_id": question.brain_id,
                    "prompt_id": self.prompt_to_use_id,
                }
            )
        )

        brain = None

        if question.brain_id:
            brain = get_brain_by_id(question.brain_id)

        return GetChatHistoryOutput(
            **{
                "chat_id": self.chat_id,
                "user_message": question.question,
                "assistant": answer,
                "message_time": new_chat.message_time,
                "prompt_title": self.prompt_to_use.title
                if self.prompt_to_use
                else None,
                "brain_name": brain.name if brain else None,
                "message_id": new_chat.message_id,
            }
        )

    # async def generate_stream(
    #     self, chat_id: UUID, question: ChatQuestion
    # ) -> AsyncIterable:
    #     history = get_chat_history(self.chat_id)
    #     callback = AsyncIteratorCallbackHandler()
    #     self.callbacks = [callback]

    #     answering_llm = self._create_llm(
    #         model=self.model, streaming=True, callbacks=self.callbacks
    #     )

    #     # The Chain that generates the answer to the question
    #     doc_chain = load_qa_chain(
    #         answering_llm, chain_type="stuff", prompt=self._create_prompt_template()
    #     )

    #     # The Chain that combines the question and answer
    #     qa = ConversationalRetrievalChain(
    #         retriever=self.vector_store.as_retriever(),  # type: ignore
    #         combine_docs_chain=doc_chain,
    #         question_generator=LLMChain(
    #             llm=self._create_llm(model=self.model), prompt=CONDENSE_QUESTION_PROMPT
    #         ),
    #         verbose=False,
    #     )

    #     transformed_history = format_chat_history(history)

    #     response_tokens = []

    #     async def wrap_done(fn: Awaitable, event: asyncio.Event):
    #         try:
    #             await fn
    #         except Exception as e:
    #             logger.error(f"Caught exception: {e}")
    #         finally:
    #             event.set()

    #     prompt_content = self.prompt_to_use.content if self.prompt_to_use else None
    #     run = asyncio.create_task(
    #         wrap_done(
    #             qa.acall(
    #                 {
    #                     "question": question.question,
    #                     "chat_history": transformed_history,
    #                     "custom_personality": prompt_content,
    #                 }
    #             ),
    #             callback.done,
    #         )
    #     )

    #     brain = None

    #     if question.brain_id:
    #         brain = get_brain_by_id(question.brain_id)

    #     streamed_chat_history = update_chat_history(
    #         CreateChatHistory(
    #             **{
    #                 "chat_id": chat_id,
    #                 "user_message": question.question,
    #                 "assistant": "",
    #                 "brain_id": question.brain_id,
    #                 "prompt_id": self.prompt_to_use_id,
    #             }
    #         )
    #     )

    #     streamed_chat_history = GetChatHistoryOutput(
    #         **{
    #             "chat_id": str(chat_id),
    #             "message_id": streamed_chat_history.message_id,
    #             "message_time": streamed_chat_history.message_time,
    #             "user_message": question.question,
    #             "assistant": "",
    #             "prompt_title": self.prompt_to_use.title
    #             if self.prompt_to_use
    #             else None,
    #             "brain_name": brain.name if brain else None,
    #         }
    #     )

    #     async for token in callback.aiter():
    #         logger.info("Token: %s", token)
    #         response_tokens.append(token)
    #         streamed_chat_history.assistant = token
    #         yield f"data: {json.dumps(streamed_chat_history.dict())}"

    #     await run
    #     assistant = "".join(response_tokens)

    #     update_message_by_id(
    #         message_id=str(streamed_chat_history.message_id),
    #         user_message=question.question,
    #         assistant=assistant,
    #     )


In [21]:
from uuid import uuid4

thing=QABaseBrainPicking(chat_id=uuid4())
thing.connect_vector_store(embeddings_model,index_name)
thing.create_prompt_template()
thing.generate_answer("What are some latch failure modes?")

2023-09-05 23:07:22,651:INFO - HTTP Request: GET https://ldoeollhxasevurjamos.supabase.co/rest/v1/chat_history?select=%2A&chat_id=eq.e754f546-4de4-4590-ae6d-21462480522b&order=message_time "HTTP/1.1 404 Not Found"


APIError: {'code': '42P01', 'details': None, 'hint': None, 'message': 'relation "public.chat_history" does not exist'}

In [ ]:
# https://github.com/StanGirard/quivr/blob/d0370ab499465ee1404d3c1d32878e8da3853441/backend/llm/prompts
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language. include the follow up instructions in the standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)
